In [2]:
from guess_number import guess_number
from numpy import random, mean

def fix_len_num(num: int, length: int) -> str:
    num = str(num)
    return '0'*(length-len(num)) + num 

def average_int_result(func, repeat=10000, print_chart=False):
    """Decorator for 

    Args:
        func (_type_): _description_
        repeat (int, optional): _description_. Defaults to 10000.
        print_chart (bool, optional): _description_. Defaults to False.
    """
    def decorator(func):
        def dec_func(*args, **kwargs):
            results = [func(*args, **kwargs) for i in range(repeat)]
            avrg_res = round(mean(results))
            
            if print_chart:
                # Printing a simple chart
                min_res = min(results) # lower bound of chart value axis
                max_res = max(results) # upper bound of chart value axis
                # Creating a dictionary:
                # keys are results returned by func();
                # values are numbers of times each result was returned.
                results = {i: results.count(i)
                            for i in range(min_res, max_res+1)}
                max_val = max(results.values()) # count of the most frequent result
                for result in results.items():
                    print(fix_len_num(result[0], 2), '-',
                            fix_len_num(result[1], 4), 
                            ('=' if result[0] == avrg_res else '-') 
                            * round(result[1]/max_val*30)) 
            return avrg_res
        return dec_func
    return decorator(func)

In [3]:
guess_number = average_int_result(guess_number, print_chart=True)
for hi in [10**i for i in range(2,6)]:
    print(f"\n1 <= number <= {hi}")
    print(f"Average number of tries is {guess_number(1, hi, bnr_srch=True)}")


1 <= number <= 100
01 - 0097 -
02 - 0213 --
03 - 0404 ---
04 - 0752 ------
05 - 1554 -------------
06 - 3316 ===========================
07 - 3664 ------------------------------
Average number of tries is 6

1 <= number <= 1000
01 - 0008 
02 - 0020 
03 - 0038 
04 - 0073 
05 - 0169 -
06 - 0323 --
07 - 0648 ----
08 - 1245 --------
09 - 2560 ================
10 - 4916 ------------------------------
Average number of tries is 9

1 <= number <= 10000
01 - 0001 
02 - 0003 
03 - 0007 
04 - 0006 
05 - 0013 
06 - 0025 
07 - 0059 
08 - 0142 -
09 - 0242 --
10 - 0528 ----
11 - 1045 --------
12 - 2071 ===============
13 - 4015 ------------------------------
14 - 1843 --------------
Average number of tries is 12

1 <= number <= 100000
06 - 0004 
07 - 0007 
08 - 0014 
09 - 0028 
10 - 0058 -
11 - 0106 -
12 - 0194 --
13 - 0412 ----
14 - 0833 -------
15 - 1680 ---------------
16 - 3228 ============================
17 - 3436 ------------------------------
Average number of tries is 16
